In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re, string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer
nltk.download('stopwords')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
file_path = 'final_nnp.csv'

In [8]:
df = pd.read_csv(file_path)

In [9]:
df.shape

(24707, 3)

In [10]:
df.groupby("rating").rating.count()

rating
negative    9055
neutral     8171
positive    7481
Name: rating, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24707 entries, 0 to 24706
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24707 non-null  int64 
 1   rating      24707 non-null  object
 2   content     24707 non-null  object
dtypes: int64(1), object(2)
memory usage: 579.2+ KB


In [12]:
df.isnull().sum()

Unnamed: 0    0
rating        0
content       0
dtype: int64

## Preprocessing

In [13]:
from bs4 import BeautifulSoup
df['content'] = df['content'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().strip())
df['content'] = df['content'].str.replace('\n', '')
df['content'][0]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11152\2034244692.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['content'] = df['content'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().strip())


"I took my seven year old to see this last night and she was fine for the first half hour or so and then she became really bored with it. After about an hour and 20 minutes she was asking if we could leave and do something else, that's not a good sign when even young girls don''t like this film.From my own personal point of view, the film is way too long for kids, the songs are borderline horrific, seriously, whoever thought putting a rap song in a film like the little mermaid was a good idea needs to be sectioned, talk about butchering a classic film score.The special effects, all I can say is, how has a film with a budget this big been made to look so poor, some of the underwater scenes look like something from a playstation 2 cut scene, they're awful.The animal characters in this, Scuttle, Sebastian and Flounder are dreadful, they look freaky, not cartoonish or charming, not even realistic, just weird and odd and they really take you out of the film whenever they're on screen.The pe

In [14]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [15]:
words = stopwords.words("english")
# words.extend(["a", "an", "the","movie","would", "could", "might", "film","shall"])
print(len(words))
remove_words = ['aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'don', "don't", 'should', "should've", 'no', 'nor', 'not',"but", 'do', 'does', 'did', 'doing']
new_list = [i for i in words if i not in remove_words]
# new_list = new_list + ["not_" + w for w in new_list]
print(new_list)
print(len(new_list))
# stopwords is inside new_list


179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'a', 'an', 'the', 'and', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'now',

In [16]:
df.head()

,Unnamed: 0,rating,content
0,19603,positive,"This movie won audience choice in Calgary, but..."
1,5345,positive,I had the honor of watching TDK during a scree...
2,9406,positive,One of the best I've ever seen.The movie had t...
3,9051,neutral,I read Stephen King's The Green Mile long befo...
4,24325,neutral,Far and away the most overrated film of all ti...


In [17]:
df.drop("Unnamed: 0", axis = 1, inplace = True)
df.head()

,rating,content
0,positive,"This movie won audience choice in Calgary, but..."
1,positive,I had the honor of watching TDK during a scree...
2,positive,One of the best I've ever seen.The movie had t...
3,neutral,I read Stephen King's The Green Mile long befo...
4,neutral,Far and away the most overrated film of all ti...


In [18]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


nltk.download('punkt')


porter_stemmer = PorterStemmer()

def remove_special_char(x):
    return re.sub("[^a-zA-Z_]", " ", x)

def stop_words_removal(x):
    clean_data = []
    for i in x.split(): #tokenize
        if i.strip().lower() not in new_list:
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

def contraction_expansion(x):
    x = re.sub(r"won\'t", "would not", x)
    x = re.sub(r"can\'t", "can not", x)
    x = re.sub(r"don\'t", "do not", x)
    x = re.sub(r"shouldn\'t", "should not", x)
    x = re.sub(r"needn\'t", "need not", x)
    x = re.sub(r"hasn\'t", "has not", x)
    x = re.sub(r"haven\'t", "have not", x)
    x = re.sub(r"weren\'t", "were not", x)
    x = re.sub(r"mightn\'t", "might not", x)
    x = re.sub(r"didn\'t", "did not", x)
    x = re.sub(r"isn\'t", "is not", x)
    x = re.sub(r"\'re", "are", x)
    x = re.sub(r"\'s", "is", x)
    x = re.sub(r"\'d", "would", x)
    x = re.sub(r"\'ll", "will", x)
    x = re.sub(r"\'t", "not", x)
    x = re.sub(r"\'ve", "have", x)
    x = re.sub(r"\'m", "am", x)
    return x



def stemming(x):
    return ' '.join([porter_stemmer.stem(word) for word in x.split()])

def cleaning(x):

    x = contraction_expansion(x)

    x = remove_special_char(x)

    x = stop_words_removal(x)


    x = stemming(x)

    return x



df["content"] = df["content"].apply(cleaning)





[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
X = df["content"]
Y = df["rating"]

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, stratify = Y, random_state = 2)


In [21]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print(X_train)
print(X_test)

  (0, 1666)	0.08346358658243398
  (0, 32209)	0.12972706394355749
  (0, 29378)	0.08951729180309223
  (0, 36286)	0.0690193673701657
  (0, 13322)	0.13066634245694608
  (0, 3914)	0.09713531390639266
  (0, 15754)	0.1719259982600924
  (0, 17770)	0.1673728226446867
  (0, 6103)	0.14670427161565253
  (0, 29928)	0.12139616793501996
  (0, 32144)	0.1673728226446867
  (0, 22121)	0.14359799647521101
  (0, 10531)	0.21713043003237753
  (0, 12540)	0.11151164578818991
  (0, 6050)	0.12333015037326224
  (0, 21038)	0.1039662012821928
  (0, 5315)	0.10986961522272846
  (0, 14770)	0.14931710459268602
  (0, 24633)	0.14096679400439222
  (0, 31707)	0.1417681459340517
  (0, 34649)	0.07968915236940756
  (0, 19079)	0.10128281677376061
  (0, 11190)	0.15965273448304676
  (0, 27396)	0.08438739036793105
  (0, 17954)	0.09033448784543881
  :	:
  (19764, 24890)	0.3376437797823649
  (19764, 7057)	0.2760098330525613
  (19764, 32557)	0.24933442756723034
  (19764, 10323)	0.19661672126610952
  (19764, 13549)	0.1605124701421545

In [22]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import LabelEncoder

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# Instantiate the SVC model with 'one-versus-one' strategy
svm_model = SVC(kernel='rbf', decision_function_shape='ovo')

# Wrap the SVM model with OneVsOneClassifier
model_one = OneVsOneClassifier(svm_model)

# Train the model
model_one.fit(X_train, Y_train_encoded)

# Evaluate the model
X_train_prediction = model_one.predict(X_train)
training_accuracy = accuracy_score(Y_train_encoded, X_train_prediction)
print("Training accuracy without cross validation: ", training_accuracy)

X_test_prediction = model_one.predict(X_test)
test_accuracy = accuracy_score(Y_test_encoded, X_test_prediction)
print("Test accuracy without cross validation: ", test_accuracy)

Training accuracy without cross validation:  0.9612952188211485
Test accuracy without cross validation:  0.7045730473492513


## With rbf Kernel

In [23]:
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.svm import SVC
# model_one = SVC(kernel= "rbf")

# model_one.fit(X_train, Y_train)


In [24]:
# X_train_prediction = model_one.predict(X_train)
# training_accuracy = accuracy_score(Y_train, X_train_prediction)
# print(training_accuracy)

In [25]:
# X_test_prediction = model_one.predict(X_test)
# test_accuracy = accuracy_score(Y_test, X_test_prediction)
# print(test_accuracy)

In [26]:
# ytest = np.array(Y_test)
# print(classification_report(ytest, model_one.predict(X_test)))
# print(confusion_matrix(ytest, model_one.predict(X_test)))

## With Linear Kernel

In [27]:
# from sklearn.svm import SVC
# model_two = SVC(kernel = "linear")
# model_two.fit(X_train, Y_train)

In [28]:
# X_train_prediction = model_two.predict(X_train)
# training_accuracy = accuracy_score(Y_train, X_train_prediction)
# print(training_accuracy)

In [29]:
# X_test_prediction = model_two.predict(X_test)
# test_accuracy = accuracy_score(Y_test, X_test_prediction)
# print(test_accuracy)

In [30]:
# ytest = np.array(Y_test)
# print(classification_report(ytest, model_two.predict(X_test)))
# print(confusion_matrix(ytest, model_two.predict(X_test)))

In [64]:
review = input("Enter review = ")
review_cleaned = cleaning(review)
print("\nCleaned Input Sentence:", review_cleaned)
review_vectorized = vectorizer.transform([review_cleaned])
predict_review_senti = model_one.predict(review_vectorized)
print("Predicted sentiment = ", predict_review_senti[0])








Enter review =  hate



Cleaned Input Sentence: hate
Predicted sentiment =  0


In [32]:
# review = input("Enter review = ")
# review_cleaned = cleaning(review)
# review_vectorized = vectorizer.transform([review_cleaned])
# predict_review_senti = model_two.predict(review_vectorized)
# print("Predicted sentiment = ", predict_review_senti[0])






## With Polynomial

In [33]:
# from sklearn.svm import SVC
# model_three = SVC(kernel = "poly" , degree=3, C=1.0)
# model_three.fit(X_train, Y_train)

In [34]:
# X_train_prediction = model_three.predict(X_train)
# training_accuracy = accuracy_score(Y_train, X_train_prediction)
# print(training_accuracy)

In [35]:
# X_test_prediction = model_three.predict(X_test)
# test_accuracy = accuracy_score(Y_test, X_test_prediction)
# print(test_accuracy)

In [36]:
# review = input("Enter review = ")
# review_cleaned = cleaning(review)
# review_vectorized = vectorizer.transform([review_cleaned])
# predict_review_senti = model_three.predict(review_vectorized)
# print("Predicted sentiment = ", predict_review_senti[0])


In [37]:
# import pickle

In [38]:
# with open('new_svm.pkl', 'wb') as model_file:
#     pickle.dump(model_one, model_file)

# # Save the vectorizer to a file
# with open('new_vectorizer.pkl', 'wb') as vectorizer_file:
#     pickle.dump(vectorizer, vectorizer_file)